In [1]:
# Import Statements
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [3]:
# Load and preprocess the dataset
df_attractions = pd.read_csv("ars.csv", encoding = "ISO-8859-1").drop_duplicates(subset=['Attraction_Name']).reset_index(drop=True)

In [5]:
# Fill missing values
df_attractions['Attraction_Name'] = df_attractions['Attraction_Name'].fillna("")
df_attractions['city'] = df_attractions['city'].fillna("")
df_attractions['subcategories'] = df_attractions['subcategories'].fillna("General")
df_attractions['rating'] = df_attractions['rating'].fillna(0).astype(float)
df_attractions['ranking'] = df_attractions['ranking'].fillna("Unknown ranking")

In [6]:
# Process the ranking column
def process_ranking(ranking):
    """Extract numeric rank from strings like '27 of 90 hotels in Lahore'."""
    match = re.search(r"(\d+) of (\d+)", ranking)
    if match:
        rank, total = map(int, match.groups())
        return rank / total  # Normalize rank to a scale of 0 to 1 (lower is better)
    return 1.0  # Default for missing or invalid rankings

df_attractions['normalized_ranking'] = df_attractions['ranking'].apply(process_ranking)

In [7]:
# Initialize TF-IDF vectorizers
tfidf_title = TfidfVectorizer(stop_words="english")
tfidf_city = TfidfVectorizer(stop_words="english")
tfidf_subcategories = TfidfVectorizer(stop_words="english")

In [8]:
# Fit TF-IDF on dataset columns
tfidf_matrix_title = tfidf_title.fit_transform(df_attractions['Attraction_Name'])
tfidf_matrix_city = tfidf_city.fit_transform(df_attractions['city'])
tfidf_matrix_category = tfidf_subcategories.fit_transform(df_attractions['subcategories'])

In [13]:
# Recommendation function
def get_recommendations(query_title, query_city, query_subcategories, df_attractions=df_attractions):
    # Transform user inputs
    query_vector_title = tfidf_title.transform([query_title])
    query_vector_city = tfidf_city.transform([query_city])
    query_vector_subcategories = tfidf_subcategories.transform([query_subcategories])
    
    # Compute cosine similarities
    title_sim_scores = cosine_similarity(query_vector_title, tfidf_matrix_title).flatten()
    city_sim_scores = cosine_similarity(query_vector_city, tfidf_matrix_city).flatten()
    subcategories_sim_scores = cosine_similarity(query_vector_subcategories, tfidf_matrix_category).flatten()
    
    # Normalize scores
    scaler = MinMaxScaler()
    title_sim_scores = scaler.fit_transform(title_sim_scores.reshape(-1, 1)).flatten()
    city_sim_scores = scaler.fit_transform(city_sim_scores.reshape(-1, 1)).flatten()
    subcategories_sim_scores = scaler.fit_transform(subcategories_sim_scores.reshape(-1, 1)).flatten()
    
    # Weighted combination of scores
    df_attractions['similarity_score'] = (
        0.3 * title_sim_scores +
        0.3 * city_sim_scores +
        0.4 * subcategories_sim_scores
    )
    
    # Final score combining similarity, rating, and ranking
    df_attractions['final_score'] = (
        0.6 * df_attractions['similarity_score'] +
        0.3 * (df_attractions['rating'] / 5) -  # Normalize ratings to a scale of 0-1
        0.1 * df_attractions['normalized_ranking']  # Penalize higher rankings
    )
    
    # Get top recommendations
    top_results = df_attractions.nlargest(10, 'final_score')

    # Initialize the ChatGroq model
    llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0.7, groq_api_key="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")
    
    # Define the system and human templates
    system = "You are a professional and user-friendly assistant that provides detailed yet concise hotel descriptions in paragraph form. Your responses should be engaging, precise, and relatable, highlighting key details provided in the variables and emphasizing user preferences in a natural, conversational style."
    human = (
        "{attraction_name} is situated in {city}, boasting a commendable rating of {rating}/5.0 and ranked {ranking}. Its aligning with user preferences for a tailored experience."
        "Your task is to craft a single, well-structured paragraph that emphasizes the details provided in the variables, presenting a catchy and personalized description that feels relatable to the user."
    )
    
    # Create the prompt template
    prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
    
    # Function to generate description using ChatGroq
    def generate_description(llm, prompt, hotel_data):
        # try:
            # Prepare the input data for the prompt
            input_data = {
                "attraction_name": hotel_data["Attraction_Name"],
                "city": hotel_data["city"],
                "rating": hotel_data["rating"],
                "ranking": hotel_data["ranking"],
                "subcategories": hotel_data["subcategories"],

            }
            
            # Create a chain from the prompt and model, and invoke it
            chain = prompt | llm
            response = chain.invoke(input_data)
            
            # Return the generated text
            return (response.content)
            # return response.get("text", "No text returned by the model.")
        # except Exception as e:
            # return f"Error generating description: {str(e)}"

    # Test the function with ChatGroq
    if not top_results.empty:
        # Generate a descriptive text for the first hotel
        first_hotel = top_results.iloc[0]
        description = generate_description(llm, prompt, first_hotel)
    
        # Return the first recommendation as text and the rest as a table
        remaining_hotels = top_results.iloc[1:][['Attraction_Name', 'id','city', 'Category', 'rating', 'ranking', 'subcategories']]
        return description, remaining_hotels
    
    return "No results found for the given query."

In [16]:
# Test the function
description, remaining_hotels = get_recommendations(
    query_title="Ocean Tower",
    query_city="Karachi",
    query_subcategories="Sights & Landmarks Observation Decks & Towers Points of Interest & Landmarks"
)

In [17]:
# Display results
print(description)
remaining_hotels

Welcome to the Ocean Tower in Karachi, a highly-rated hotel that's sure to make your stay in the city memorable. With a fantastic rating of 4.0 out of 5.0 and ranked an impressive #19 out of 111 things to do in Karachi, you can trust that you're in for a treat. This hotel has been tailored to meet your preferences, ensuring a personalized experience that will make you feel right at home. Located in the heart of Karachi, the Ocean Tower offers easy access to the city's top attractions, making it the perfect base for your explorations. Whether you're in town for business or pleasure, you'll appreciate the hotel's comfortable accommodations, excellent amenities, and unbeatable location. So come and experience the best of Karachi at the Ocean Tower - we can't wait to welcome you!


,Attraction_Name,id,city,Category,rating,ranking,subcategories
51,"Merewether Clock Tower, Karachi",9811631,karachi,attraction,4.5,#23 of 111 things to do in Karachi,Sights & Landmarks Points of Interest & Landma...
42,"Koel Gallery, Karachi",6200580,karachi,attraction,4.5,#26 of 111 things to do in Karachi,Sights & Landmarks Points of Interest & Landmarks
40,"National Academy Of Performing Arts, Karachi",6213077,karachi,attraction,4.5,#28 of 111 things to do in Karachi,Sights & Landmarks Points of Interest & Landmarks
72,"Art Chowk, Karachi",7317593,karachi,attraction,5.0,#73 of 111 things to do in Karachi,Sights & Landmarks Points of Interest & Landmarks
12,"Empress Market, Karachi",553629,karachi,attraction,4.0,#8 of 111 things to do in Karachi,Sights & Landmarks Points of Interest & Landmarks
19,"Boat Basin, Karachi",6998658,karachi,attraction,4.0,#20 of 111 things to do in Karachi,Sights & Landmarks Points of Interest & Landmarks
10,"Tdf Ghar, Karachi",15840966,karachi,attraction,5.0,#11 of 111 things to do in Karachi,Sights & Landmarks Architectural Buildings His...
65,"Eduljee Dinshaw Road, Karachi",9827944,karachi,attraction,5.0,#59 of 111 things to do in Karachi,Sights & Landmarks Scenic Walking Areas Points...
50,"Chaukhandi Tombs, Karachi",13327193,karachi,attraction,4.5,#31 of 111 things to do in Karachi,Sights & Landmarks Points of Interest & Landma...


In [18]:
related_hotels_to = int(input("Enter selected attraction id: "))

Enter selected attraction id:  6998658


In [20]:
selected_hotel_details = df_attractions.loc[df_attractions['id'] == related_hotels_to]

In [21]:
selected_hotel_details.iloc[0]["city"]

'karachi'

In [24]:
description, remaining_hotels = get_recommendations(
    query_title="",
    query_city=selected_hotel_details.iloc[0]["city"],
    query_subcategories=selected_hotel_details.iloc[0]["subcategories"]
)

In [25]:
# Display results for related hotels
print(description)
remaining_hotels

Welcome to the Boat Basin in Karachi, a true gem of the city that has earned a stellar rating of 4.0 out of 5.0 and is ranked #20 out of 111 things to do in Karachi. If you're looking for a personalized and tailored experience, then this is the place to be. The Boat Basin is not just a hotel, it's a destination that offers a little bit of everything for everyone. Whether you're a foodie looking to explore the local cuisine, an adventure seeker wanting to discover the city's hidden gems, or someone who simply wants to relax and unwind, the Boat Basin has got you covered. With its prime location, you'll be close to all the action, yet far enough to enjoy some peace and quiet. Come and experience the magic of Karachi at the Boat Basin, where unforgettable memories are made.


,Attraction_Name,id,city,Category,rating,ranking,subcategories
42,"Koel Gallery, Karachi",6200580,karachi,attraction,4.5,#26 of 111 things to do in Karachi,Sights & Landmarks Points of Interest & Landmarks
40,"National Academy Of Performing Arts, Karachi",6213077,karachi,attraction,4.5,#28 of 111 things to do in Karachi,Sights & Landmarks Points of Interest & Landmarks
72,"Art Chowk, Karachi",7317593,karachi,attraction,5.0,#73 of 111 things to do in Karachi,Sights & Landmarks Points of Interest & Landmarks
12,"Empress Market, Karachi",553629,karachi,attraction,4.0,#8 of 111 things to do in Karachi,Sights & Landmarks Points of Interest & Landmarks
37,"I.I. Chundriger Rd, Karachi",553616,karachi,attraction,3.5,#36 of 111 things to do in Karachi,Sights & Landmarks Points of Interest & Landmarks
77,"Canvas Gallery, Karachi",6800583,karachi,attraction,4.0,#74 of 111 things to do in Karachi,Sights & Landmarks Points of Interest & Landmarks
423,"Amin Gulgee Gallery, Karachi",7317628,karachi,attraction,4.5,No ranking,Sights & Landmarks Points of Interest & Landmarks
65,"Eduljee Dinshaw Road, Karachi",9827944,karachi,attraction,5.0,#59 of 111 things to do in Karachi,Sights & Landmarks Scenic Walking Areas Points...
10,"Tdf Ghar, Karachi",15840966,karachi,attraction,5.0,#11 of 111 things to do in Karachi,Sights & Landmarks Architectural Buildings His...
